In [8]:
from model2 import network, input_dim, hidden_dim, num_layers, char2oh, vocab 
import torch
from torch.autograd import Variable
from sys import stdout
    
def encode(letter):
    """One-hot encoding of letter"""
    return char2oh[letter]

def decode(tensor):
    """Letter from one-hot encoding"""
    index = tensor.view(-1).max(0)[1]
    return vocab[index.item()]

prompt = "The next big invention " #feed in a input prompt
speech_len = 1000

In [10]:
if __name__=="__main__":
    
    speaker = network(91, 128, 2)
    speaker.load_state_dict(torch.load('./checkpoints/e10_3.0835.pth',map_location=torch.device('cpu')))
    speaker.batch_size = 1
   
    speaker.reset()
    
    temperature = 0.55  
    
    for letter in prompt:
        ip = Variable(encode(letter).view(1,1,-1), volatile=True)
        output = decode(speaker(ip))
        stdout.write(letter)
    
    for counter in range(speech_len):
        ip = Variable(encode(output).view(1,1,-1), volatile=True)
        output_dist = speaker(ip).data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        output = vocab[top_i]
        stdout.write(output)

The next big invention KKKKK@KKKKKMKmTTKPKMKKL%yKKK<DKKLKmPKmKLKK4aK<%KKPKKLM%DMT@K)@4aMmKLKKKPK#PWK%KKK4KKPmP%@%KKKKPMKKMKKKMKMKLM@PKMhKm4K%PKMLKmKM)KL4KMPKLPKKKyMKDMyMKK#mhT0KvTmKPKKMPK%wTKLM4mKKmL<#KW#PKmL)KLKKKM0@MmmWKMKKKKPK14KM#KLKPTK4yK@L4lTTLK<@MPKLMK)KwMTyhKKwKKKPm%Mm,KM4MK4KK#%PKKK%4KKKMMK,mK%KMKTKmPLK@<TK@Pm#KyyT#KKK%K4%MKKKTK#P%4KM4K#KK#K@KKK<KKyK@KKMm<yMMP%MmKTKPMMMmKaK#LyKKLPdKLPW%4K%y#@MKKmKPWMKKLLMK@mP%KPKmPMmowPM4lmKKKKP%P4KKKPK4KmKmKLMMTMTKMKK41K#@KPLKgKKKMP)KP%KKmTmKPMLM4KKPMLmK%PKKK#PyPPMPK%KMMPMK<<KKKKPPP4KaTKTKMPWaM%K,PPyKK4gmKh%mP<TK4KL4TwKKw%KTK%T#%m%lMK4PK4MP%%K#MPMy#PKL%KPMMmK4LKLMK%m4P#TKP,KK@P4#LKKPmKTP#%PmMM4KKPm#MPKMmKP%@KLaKM@KK<MKKTKKmLKKMKKKP#mLKmMKKP#KMMTmKKK)MKKKLlPT@MMKKKmKPKKKPmLkKKKM%K0KP<KMK<K@KmKmKy@LKmMLKKKKKKKKKKKK%TK4KDP@K%KKMKPKPKmKK<KKKKKKKmPK)KKKKK%K4KlMKKMWKKKKKPMKK4#KmoKK@M%KPMK41KLKKMKKKKKyKK#K@KKKPKKTKl@@PPMKy1K#KMK4KMPPLK6KPmKMWKPLLKK#TKLKKmKMKK4M#%MPP@KPKLKMKKMKPM4%)L4KPKKLMK%WKK%KKMmPK%%KKmy@KmKKMKKKKKKKKKPKP4<L@MLKMKP#MK@KKKPK#yK%KK

/var/folders/4l/skxd6ygn2_xdqybw7hyxtc9h0000gn/T/ipykernel_14247/231301665.py:12: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  ip = Variable(encode(letter).view(1,1,-1), volatile=True)
/var/folders/4l/skxd6ygn2_xdqybw7hyxtc9h0000gn/T/ipykernel_14247/231301665.py:17: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  ip = Variable(encode(output).view(1,1,-1), volatile=True)


In [5]:
import torch

if __name__ == "__main__":
    
    # Initialize the speaker network
    speaker = network(91, 128, 2)
    
    # Load model weights onto the CPU
    speaker.load_state_dict(torch.load('./checkpoints/e10_3.0805.pth', map_location=torch.device('cpu')))
    speaker.batch_size = 1
   
    # Reset the speaker (ensure the network is initialized properly)
    speaker.reset()
    
    # Set the temperature for sampling
    temperature = 0.55  
    
    # Assuming 'prompt' is defined elsewhere
    for letter in prompt:
        ip = torch.tensor(encode(letter).view(1, 1, -1), dtype=torch.float32)  # Convert to tensor
        output = decode(speaker(ip))
        stdout.write(letter)
    
    # Assuming 'speech_len' is defined elsewhere
    for counter in range(speech_len):
        ip = torch.tensor(encode(output).view(1, 1, -1), dtype=torch.float32)  # Convert to tensor
        output_dist = speaker(ip).view(-1).div(temperature).exp()  # Ensure it's a tensor
        top_i = torch.multinomial(output_dist, 1)[0]
        
        output = vocab[top_i]  # Assuming vocab is defined elsewhere
        stdout.write(output)


/var/folders/4l/skxd6ygn2_xdqybw7hyxtc9h0000gn/T/ipykernel_14247/1846499860.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ip = torch.tensor(encode(letter).view(1, 1, -1), dtype=torch.float32)  # Convert to tensor


IndexError: invalid index of a 0-dim tensor. Use `tensor.item()` in Python or `tensor.item<T>()` in C++ to convert a 0-dim tensor to a number